# Recurrent Neural Networks

## Import Pacakges

In [438]:
import tensorflow as tf
import torch
from torch import nn
from torch.utils import data
from torch.autograd import Variable

import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

plt.rcParams["figure.figsize"] = (8, 8)

In [2]:
print(tf.__version__)

2.8.0


In [3]:
print(torch.__version__)

1.8.0


* https://datascience-enthusiast.com/DL/Building_a_Recurrent_Neural_Network-Step_by_Step_v1.html
* https://www.tensorflow.org/tutorials/structured_data/time_series
* https://d2l.ai/chapter_recurrent-neural-networks/sequence.html
* https://towardsdatascience.com/building-rnn-lstm-and-gru-for-time-series-using-pytorch-a46e5b094e7b
* https://colah.github.io/posts/2015-08-Understanding-LSTMs/
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/
* https://towardsdatascience.com/all-you-need-to-know-about-rnns-e514f0b00c7c

## Step by Step Run Through

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [410]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### START CODE HERE ### (≈2 lines)
    # compute next activation state using the formula given above
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    # compute output of the current cell using the formula given above
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    ### END CODE HERE ###
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

def rnn_cell_backward(da_next, cache):
    """
    Implements the backward pass for the RNN-cell (single time-step).

    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradients of input data, of shape (n_x, m)
                        da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dba -- Gradients of bias vector, of shape (n_a, 1)
    """
    
    # Retrieve values from cache
    (a_next, a_prev, xt, parameters) = cache
    
    # Retrieve values from parameters
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]

    ### START CODE HERE ###
    # compute the gradient of tanh with respect to a_next (≈1 line)
    dtanh = (1-a_next*a_next)*da_next

    # compute the gradient of the loss with respect to Wax (≈2 lines)
    dxt = np.dot(Wax.T,  dtanh)
    dWax = np.dot(dtanh,xt.T)

    # compute the gradient with respect to Waa (≈2 lines)
    da_prev = np.dot(Waa.T, dtanh)  
    dWaa = np.dot( dtanh,a_prev.T)

    # compute the gradient with respect to b (≈1 line)
    dba = np.sum( dtanh,keepdims=True,axis=-1)

    ### END CODE HERE ###
    
    # Store the gradients in a python dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}
    
    return gradients

In [454]:
# sequence = 'abcabcabcabcabc'
sequence = 'abbcacbbcacbbca' * 10
sequence = np.array([char for char in sequence]).reshape(-1, 1)

In [200]:
ohe_enc = OneHotEncoder()
ohe_enc.fit(sequence)
sequence_ohe = ohe_enc.transform(sequence).toarray()

In [216]:
sequence_ohe.shape

(150, 3)

In [497]:
def get_ts_X_y(seq, batch_size, seq_len, num_features, lookahead=1):
    X = np.zeros((batch_size, seq_len, num_features))
    y = np.zeros((batch_size, num_features))
    for i in range(batch_size):
        start_idx = np.random.choice(range(len(seq) - seq_len - lookahead))
        x_obs = seq[start_idx : start_idx + seq_len]
        y_obs = seq[start_idx + seq_len + lookahead]
        X[i] = x_obs
        y[i] = y_obs
    print(X.shape, y.shape)
    return X, y

In [498]:
num_features = len(sequence_ohe[0])
seq_len = 15
batch_size = 32
X, y = get_ts_X_y(sequence_ohe, batch_size, seq_len, num_features)

(32, 15, 3) (32, 3)


In [496]:
x = np.swapaxes(np.swapaxes(X, 0, 2), 1, 2)
x.shape

(3, 32, 15)

In [378]:
np.random.seed(1)
hidden_state_size = 4
num_input_features = 3
num_output_features = 3

xt = sequence_ohe[0]
a_prev = np.random.randn(hidden_state_size, )
Waa = np.random.randn(hidden_state_size, hidden_state_size)
Wax = np.random.randn(hidden_state_size, num_input_features)
Wya = np.random.randn(num_output_features, hidden_state_size)
ba = np.random.randn(hidden_state_size, )
by = np.random.randn(num_output_features, )
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[2] = ", a_next[2])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[2] =  0.34572473052766445
a_next.shape =  (4,)
yt_pred[1] = 0.5836545827347673
yt_pred.shape =  (3,)


In [395]:
def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y" with zeros (≈2 lines)
    a = np.zeros([n_a,m,T_x])
    y_pred = np.zeros([n_y,m,T_x])
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache (≈1 line)
        xt = x[:, :, t]
        a_next, yt_pred, cache = rnn_cell_forward(xt, a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:, :, t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y_pred[:, :, t] = yt_pred
        # Append "cache" to "caches" (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

def rnn_backward(da, caches):
    """
    Implement the backward pass for a RNN over an entire sequence of input data.

    Arguments:
    da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient w.r.t. the input data, numpy-array of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (n_a, m)
                        dWax -- Gradient w.r.t the input's weight matrix, numpy-array of shape (n_a, n_x)
                        dWaa -- Gradient w.r.t the hidden state's weight matrix, numpy array of shape (n_a, n_a)
                        dba -- Gradient w.r.t the bias, of shape (n_a, 1)
    """
        
    ### START CODE HERE ###
    
    # Retrieve values from the first cache (t=1) of caches (≈2 lines)
    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[0]
    
    # Retrieve dimensions from da's and x1's shapes (≈2 lines)
    n_a, m, T_x = da.shape
    n_x, m = x1.shape 
    
    # initialize the gradients with the right sizes (≈6 lines)
    dx = np.zeros((n_x, m, T_x)) 
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1)) 
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))  
    
    # Loop through all the time steps
    for t in reversed(range(T_x)):
        # Compute gradients at time step t. Choose wisely the "da_next" and the "cache" to use in the backward propagation step. (≈1 line)
        gradients = rnn_cell_backward(da[:, :, t] + da_prevt, caches[t])
        # Retrieve derivatives from gradients (≈ 1 line)
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dWaa"], gradients["dba"]
        # Increment global derivatives w.r.t parameters by adding their derivative at time-step t (≈4 lines)
        dx[:, :, t] = dxt  
        dWax += dWaxt  
        dWaa += dWaat  
        dba += dbat  
        
    # Set da0 to the gradient of a which has been backpropagated through all time-steps (≈1 line) 
    da0 = da_prevt
    ### END CODE HERE ###

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
    
    return gradients

In [411]:
np.random.seed(1)
hidden_state_size = 10
num_input_features = len(x)
num_output_features = len(x)
num_time_steps = len(x[0][0])

# x = sequence_ohe.T.reshape((num_input_features, 1, num_time_steps))
a0 = np.random.randn(hidden_state_size, batch_size)
Waa = np.random.randn(hidden_state_size, hidden_state_size)
Wax = np.random.randn(hidden_state_size, num_input_features)
Wya = np.random.randn(num_output_features, hidden_state_size)
ba = np.random.randn(hidden_state_size, batch_size)
by = np.random.randn(num_output_features, batch_size)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a.shape = ", a.shape)
print("y_pred.shape = ", y_pred.shape)
print("len(caches) = ", len(caches))

da = np.random.randn(hidden_state_size, batch_size, num_time_steps)
gradients_tmp = rnn_backward(da, caches)

a.shape =  (10, 32, 15)
y_pred.shape =  (3, 32, 15)
len(caches) =  2


In [412]:
pred_class = np.argmax(y_pred, axis=0)
act_class = np.argmax(x, axis=0)

In [415]:
act_class

array([[2, 1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0],
       [2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1],
       [0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 2],
       [1, 2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1],
       [2, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 2, 0],
       [2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 1, 1],
       [0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 2, 0],
       [2, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 2, 0],
       [2, 1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0],
       [1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2],
       [2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1],
       [1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 1],
       [2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1],
       [1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1],
       [1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1],
       [0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 2, 0],
       [2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1],
       [2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 1, 1],
       [0, 2, 1, 1, 2, 0, 2,

In [416]:
pred_class

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 0, 2, 2, 0, 0, 0, 2, 1, 2, 0, 0, 2, 0, 2],
       [1, 1, 1, 1, 2, 0, 0, 1, 1, 2, 2, 0, 1, 1, 2],
       [2, 1, 0, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 0, 2],
       [1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0],
       [2, 2, 1, 1, 2, 2, 2, 2, 0, 1, 1, 2, 2, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 0, 2],
       [2, 2, 0, 2, 0, 0, 2, 0, 1, 2, 2, 0, 0, 2, 0],
       [2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 0, 2],
       [0, 1, 1, 2, 2, 2, 0, 0, 1, 1, 2, 1, 0, 2, 1],
       [1, 2, 2, 0, 1, 1, 2, 0, 2, 0, 0, 1, 0, 2, 0],
       [1, 1, 1, 1, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0],
       [0, 2, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0, 2, 0],
       [1, 1, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1],
       [0, 0, 1, 2, 1, 1, 1,

In [420]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu().

    Defined in :numref:`sec_use_gpu`"""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

In [ ]:
def predict_ch8(prefix, num_preds, net, vocab):
    """Generate new characters following the `prefix`.

    Defined in :numref:`sec_rnn_scratch`"""
    state = net.begin_state(batch_size=1, dtype=tf.float32)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: d2l.reshape(d2l.tensor([outputs[-1]]), (1, 1)).numpy()
    for y in prefix[1:]:  # Warm-up period
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # Predict `num_preds` steps
        y, state = net(get_input(), state)
        outputs.append(int(y.numpy().argmax(axis=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [417]:
class RNNModel(tf.keras.layers.Layer):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, state):
        X = tf.one_hot(tf.transpose(inputs), self.vocab_size)
        # Later RNN like `tf.keras.layers.LSTMCell` return more than two values
        Y, *state = self.rnn(X, state)
        output = self.dense(tf.reshape(Y, (-1, Y.shape[-1])))
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.cell.get_initial_state(*args, **kwargs)

In [423]:
num_hiddens = 256
rnn_cell = tf.keras.layers.SimpleRNNCell(num_hiddens,
    kernel_initializer='glorot_uniform')
rnn_layer = tf.keras.layers.RNN(rnn_cell, time_major=True,
    return_sequences=True, return_state=True)

state = rnn_cell.get_initial_state(batch_size=batch_size, dtype=tf.float32)
state.shape

TensorShape([32, 256])

In [542]:
device_name = try_gpu()._device_name
strategy = tf.distribute.OneDeviceStrategy(device_name)
with strategy.scope():
    net = RNNModel(rnn_layer, vocab_size=num_input_features)

In [558]:
n_steps = 15
n_features = 3

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
    # tf.keras.layers.Softmax(3)
    tf.keras.layers.Dense(n_features, activation=tf.nn.softmax)
])

In [559]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 50)                10800     
                                                                 
 dense_11 (Dense)            (None, 3)                 153       
                                                                 
Total params: 10,953
Trainable params: 10,953
Non-trainable params: 0
_________________________________________________________________


In [560]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

For tf.keras.layers.LSTM model we need inputs with shape `[batch, timesteps, feature]`

In [561]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
history = model.fit(X, y, epochs=100, verbose=1, callbacks=[callback])

Epoch 1/100


2022-05-12 08:53:52.412147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 750ms/step - loss: 1.1000 - accuracy: 0.3125
Epoch 2/100
1/1 [==============================] - 0s 141ms/step - loss: 1.0952 - accuracy: 0.3125
Epoch 3/100
1/1 [==============================] - 0s 149ms/step - loss: 1.0906 - accuracy: 0.3125
Epoch 4/100
1/1 [==============================] - 0s 140ms/step - loss: 1.0860 - accuracy: 0.3125
Epoch 5/100
1/1 [==============================] - 0s 147ms/step - loss: 1.0817 - accuracy: 0.3125
Epoch 6/100
1/1 [==============================] - 0s 165ms/step - loss: 1.0774 - accuracy: 0.5312
Epoch 7/100
1/1 [==============================] - 0s 147ms/step - loss: 1.0733 - accuracy: 0.7188
Epoch 8/100
1/1 [==============================] - 0s 142ms/step - loss: 1.0693 - accuracy: 0.7188
Epoch 9/100
1/1 [==============================] - 0s 143ms/step - loss: 1.0656 - accuracy: 0.7188
Epoch 10/100
1/1 [==============================] - 0s 141ms/step - loss: 1.0618 - accuracy: 0.7188
Epoch 11/100
1/1 [===

In [562]:
test_data = X # X[0].reshape(-1, n_steps, n_features)
y_pred = model.predict(test_data, verbose=1)
y_pred.round(2)

1/1 [==============================] - 0s 144ms/step


2022-05-12 08:54:09.226157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.15, 0.  , 0.85],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.93, 0.  , 0.07],
       [1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.13, 0.86],
       [1.  , 0.  , 0.  ],
       [0.09, 0.01, 0.9 ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.95, 0.05],
       [0.93, 0.  , 0.07],
       [0.  , 0.95, 0.05],
       [0.15, 0.  , 0.85],
       [0.09, 0.01, 0.9 ],
       [1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.15, 0.  , 0.85],
       [0.  , 1.  , 0.  ],
       [0.  , 0.95, 0.05],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.39, 0.  , 0.61],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  ]], dtype=float32)

In [551]:
y

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])